In [1]:
from langchain.chat_models import ChatOpenAI #chatgpt 3.5 turbo
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [2]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [6]:
template = ChatPromptTemplate.from_messages(
    [
    (
        "system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else.",
     ),
    ("human", "{question}"),
    ]
)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [7]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items" :5,
    "question" : "What are the pokemons?",
})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']